In [1]:
from collections import Counter
import numpy as np
import pandas as pd

from sklearn.model_selection import GridSearchCV, StratifiedKFold, train_test_split
from sklearn.metrics import recall_score, precision_score, f1_score, accuracy_score, roc_auc_score
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingClassifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

import xgboost as xgb

In [2]:
# uncomment in colab
# !wget https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/telecom_churn.csv

In [3]:
raw_df = pd.read_csv('./telecom_churn.csv')

Each row represents a customer, each column contains customer’s attributes described on the column Metadata.

The are no missing values in data set.

In [4]:
raw_df.head()

,State,Account length,Area code,International plan,Voice mail plan,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,Total night charge,Total intl minutes,Total intl calls,Total intl charge,Customer service calls,Churn
0,KS,128,415,No,Yes,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,OH,107,415,No,Yes,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,NJ,137,415,No,No,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,OH,84,408,Yes,No,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,OK,75,415,Yes,No,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


In [5]:
# check for NaNs
raw_df.isnull().values.any()

False

## Dataset Attributes:

- _Churn_ - Indicates whether the customer churned

In [6]:
print('Number of duplicated values in training dataset: ', raw_df.duplicated().sum())

Number of duplicated values in training dataset:  0


In [7]:
# distinction is based on the number of different values in the column
columns = list(raw_df.columns)

categoric_columns = ['State', "Area code", "International plan", "Voice mail plan"]
numeric_columns = list(set(columns) - set(categoric_columns) - {"Churn"})
categoric_columns

['State', 'Area code', 'International plan', 'Voice mail plan']

In [8]:
categoric_columns, numeric_columns

(['State', 'Area code', 'International plan', 'Voice mail plan'],
 ['Total day calls',
  'Total night charge',
  'Number vmail messages',
  'Total eve calls',
  'Total day minutes',
  'Total day charge',
  'Customer service calls',
  'Total eve minutes',
  'Total intl charge',
  'Total night calls',
  'Account length',
  'Total eve charge',
  'Total intl minutes',
  'Total intl calls',
  'Total night minutes'])

In [9]:
for col in categoric_columns:
    print(col)
    print(raw_df[col].values)

State
['KS' 'OH' 'NJ' ... 'RI' 'CT' 'TN']
Area code
[415 415 415 ... 510 510 415]
International plan
['No' 'No' 'No' ... 'No' 'Yes' 'No']
Voice mail plan
['Yes' 'Yes' 'No' ... 'No' 'No' 'Yes']


In [10]:
# convert area code to String so it can be processed with Pandas' Label encoder (Dummy encoder)
raw_df["Area code"] = raw_df["Area code"].astype(str)
# perform one-hot encoding for the dataset
df_onehot = pd.get_dummies(raw_df, drop_first=True)
df_onehot.head()

,Account length,Number vmail messages,Total day minutes,Total day calls,Total day charge,Total eve minutes,Total eve calls,Total eve charge,Total night minutes,Total night calls,...,State_VA,State_VT,State_WA,State_WI,State_WV,State_WY,Area code_415,Area code_510,International plan_Yes,Voice mail plan_Yes
0,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,...,0,0,0,0,0,0,1,0,0,1
1,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,...,0,0,0,0,0,0,1,0,0,1
2,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,...,0,0,0,0,0,0,1,0,0,0
3,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,...,0,0,0,0,0,0,0,0,1,0
4,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,...,0,0,0,0,0,0,1,0,1,0


In [11]:
np.unique(df_onehot["Churn"].values)

array([False,  True])

In [12]:
df_X = df_onehot.drop('Churn', axis=1)
feature_names = df_X.columns
X = df_X.values.astype(np.float32)
y = df_onehot['Churn'].values.astype(np.float32)
X.shape, y.shape

((3333, 69), (3333,))

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size = 0.2, random_state = 42)

In [14]:
# make sure train/test labels are stratified
Counter(y_train)
counted_train, counted_test = Counter(y_train), Counter(y_test)
churn_ratio_train = counted_train[1] / len(y_train)
churn_ratio_test = counted_test[1] / len(y_test)
print(f"Train and test churn ratios: {churn_ratio_train:.4f} and {churn_ratio_test:.4f}")

Train and test churn ratios: 0.1448 and 0.1454


# Building a model
## Base learner analysis
Let's build best possible base models using grid search for:

- Logistic Regression
- kNN
- SVM classifier
- Random Forest
- Gradient Boosting

In [15]:
# let's fix stratified K-Fold split
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
grid_search_params = {
    "scoring": "roc_auc", # we have highly imbalanced classes,
    "verbose": 1,    # we wanna see some logs
    "n_jobs": -1,    # we wanna utilize multiprocessing for speedup
    "cv": skf,       # we wanna run cross-validation over predefined folds from 'skf'
}

## Logistic Regression

In [16]:
# logistic regression
parameters_logreg = {
    'logreg__C': np.logspace(-5, 5, 11),
    'logreg__penalty': ['l1', 'l2'],
    # 'fit_intercept': [False, True]
}
model = Pipeline([
    ('scaler', StandardScaler()),
    ('logreg', LogisticRegression(random_state=42, solver='liblinear'))
])
gcv_logreg = GridSearchCV(model, parameters_logreg, **grid_search_params)
gcv_logreg.fit(X_train, y_train)

Fitting 5 folds for each of 22 candidates, totalling 110 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('logreg',
                                        LogisticRegression(random_state=42,
                                                           solver='liblinear'))]),
             n_jobs=-1,
             param_grid={'logreg__C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05]),
                         'logreg__penalty': ['l1', 'l2']},
             scoring='roc_auc', verbose=1)

In [17]:
best_logreg = gcv_logreg.best_estimator_
best_params_logreg = gcv_logreg.best_params_
best_params_logreg

{'logreg__C': 0.1, 'logreg__penalty': 'l1'}

## k-Nearest Neighbor classifier

In [18]:
# kNN
parameters_knn = {
    'knn__n_neighbors': [2, 3, 5, 7, 9],
    'knn__weights': ['uniform', 'distance'],
}
model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])
gcv_knn = GridSearchCV(model, parameters_knn, **grid_search_params)
gcv_knn.fit(X_train, y_train)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('knn', KNeighborsClassifier())]),
             n_jobs=-1,
             param_grid={'knn__n_neighbors': [2, 3, 5, 7, 9],
                         'knn__weights': ['uniform', 'distance']},
             scoring='roc_auc', verbose=1)

In [19]:
best_knn = gcv_knn.best_estimator_
best_params_knn = gcv_knn.best_params_
best_params_knn

{'knn__n_neighbors': 9, 'knn__weights': 'distance'}

## Support Vector Machine Classifier

In [20]:
# support vector classifier (SVC)
parameters_svc = {
    'svc__C': np.logspace(-4, 4, 9),
    'svc__kernel': ['poly', 'rbf'],
}
model = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True))
])
gcv_svc = GridSearchCV(model, parameters_svc, **grid_search_params)
gcv_svc.fit(X_train, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('svc', SVC(probability=True))]),
             n_jobs=-1,
             param_grid={'svc__C': array([1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03,
       1.e+04]),
                         'svc__kernel': ['poly', 'rbf']},
             scoring='roc_auc', verbose=1)

In [21]:
best_svc = gcv_svc.best_estimator_
best_params_svc = gcv_svc.best_params_
best_params_svc

{'svc__C': 1.0, 'svc__kernel': 'rbf'}

## Random Forest

In [22]:
# Random Forest
parameters_rfc = {
    'max_features': [10, 20, 30],
    'min_samples_leaf': [3, 5, 7],
    'max_depth': [10, 20, 25],
    'n_estimators': [100, 150, 200]
}
model = RandomForestClassifier(random_state=42, n_jobs=-1, oob_score=True)
gcv_rfc = GridSearchCV(model, parameters_rfc, **grid_search_params)
gcv_rfc.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=RandomForestClassifier(n_jobs=-1, oob_score=True,
                                              random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [10, 20, 25],
                         'max_features': [10, 20, 30],
                         'min_samples_leaf': [3, 5, 7],
                         'n_estimators': [100, 150, 200]},
             scoring='roc_auc', verbose=1)

In [23]:
best_rfc = gcv_rfc.best_estimator_
best_params_rfc = gcv_rfc.best_params_
best_params_rfc

{'max_depth': 10,
 'max_features': 10,
 'min_samples_leaf': 3,
 'n_estimators': 100}

## Gradient Boosting from Scikit-learn

In [24]:
# Gradient Boosting
parameters_gb = {
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.05, 0.1, 0.3],
    'min_samples_leaf': [1, 3, 5],
    'max_depth': [2, 3, 4],
}
model = GradientBoostingClassifier(random_state=42)
gcv_gb = GridSearchCV(model, parameters_gb, **grid_search_params)
gcv_gb.fit(X_train, y_train)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=GradientBoostingClassifier(random_state=42), n_jobs=-1,
             param_grid={'learning_rate': [0.05, 0.1, 0.3],
                         'max_depth': [2, 3, 4], 'min_samples_leaf': [1, 3, 5],
                         'n_estimators': [100, 150, 200]},
             scoring='roc_auc', verbose=1)

In [25]:
best_gb = gcv_gb.best_estimator_
best_params_gb = gcv_gb.best_params_
best_params_gb

{'learning_rate': 0.05,
 'max_depth': 4,
 'min_samples_leaf': 5,
 'n_estimators': 100}

In [26]:
# bonus area: do the same for XGBoost classifier
xgb_model = xgb.XGBClassifier(objective = "binary:logistic", random_state=42)
params_xgboost = {
    'n_estimators': [100, 150, 200],
    'max_depth': [2, 3, 4],
    'eta': np.arange(0.05, 0.26, 0.05), # learning rate
}
gcv_xgb = GridSearchCV(xgb_model, param_grid = params_xgboost, **grid_search_params)
gcv_xgb.fit(X_train, y_train)

Fitting 5 folds for each of 45 candidates, totalling 225 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_typ...
                                     max_cat_to_onehot=None,
                                     max_delta_step=None, max_depth=None,
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=None,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=42, ...),
             n_jobs=-1,
             param_grid={'eta': array([0.05, 0.1 , 0.15, 0.2 , 0.25]),
                         'max_depth': [2, 3, 4],
                         'n_estimators': [100, 150, 200]},
             scoring='roc_auc', verbose=1)

In [27]:
best_xgb = gcv_xgb.best_estimator_
best_params_xgb = gcv_xgb.best_params_
best_params_xgb

{'eta': 0.05, 'max_depth': 4, 'n_estimators': 100}

## Let's evaluate the resulting best models on the test set

In [28]:
# let's put all our fitted estimators with best parameters in a list
estimators = [
    ("knn", best_knn),
    ("logreg", best_logreg),
    ("svm", best_svc),
    ("forest", best_rfc),
    ("gbc", best_gb),
    ("xgb", best_xgb),
]

In [29]:
# helper function for better error analysis
def compute_eer(fpr, tpr, thresholds):
    """ Returns equal error rate (EER) and the corresponding threshold. """
    fnr = 1-tpr
    abs_diffs = np.abs(fpr - fnr)
    min_index = np.argmin(abs_diffs)
    eer = np.mean((fpr[min_index], fnr[min_index]))
    return eer, thresholds[min_index]

# estimate many metrics for classification report
def get_binary_classification_metrics(estimator, X_data, y_gt):
    y_pred_probas = estimator.predict_proba(X_data)
    proba_class_1 = y_pred_probas[:, 1]
    
    y_pred = np.int32(proba_class_1 > 0.5) # usually equal to estimator.predict(X_data)
    f1 = f1_score(y_pred, y_gt)
    precision = precision_score(y_pred, y_gt)
    recall = recall_score(y_pred, y_gt)
    
    roc_auc = roc_auc_score(y_gt, proba_class_1)
    fpr, tpr, thresholds = roc_curve(y_gt, proba_class_1)
    eer, eer_thresh = compute_eer(fpr, tpr, thresholds)
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1,
        "roc_auc": roc_auc,
        "eer": eer,
        "eer_thresh": eer_thresh
    }

def print_report(metric_dict, model_name=""):
    if len(model_name):
        print(f"model: {model_name:10} || ", end="")
    # metrics for imbalanced classes
    f1, precision, recall = metric_dict["f1"], metric_dict["precision"], metric_dict["recall"]
    # ROC-curve-based metrics
    roc_auc = metric_dict["roc_auc"]
    eer, eer_thresh = metric_dict["eer"], metric_dict["eer_thresh"]
    print(f"f1: {f1:.3f} | prec: {precision:.3f} recall: {recall:.3f} | ", end="")
    print(f"ROC AUC: {roc_auc:.3f} | EER: {eer:.3f} @ th={eer_thresh:.3f}")

In [30]:
from sklearn.metrics import roc_curve
# let's evaluate our best selected model one-by-one on the holdout test set
for model_name, model in estimators:
    model_metrics = get_binary_classification_metrics(model, X_test, y_test)
    
    print_report(model_metrics, model_name=model_name)

model: knn        || f1: 0.020 | prec: 0.010 recall: 1.000 | ROC AUC: 0.684 | EER: 0.367 @ th=0.110
model: logreg     || f1: 0.292 | prec: 0.206 recall: 0.500 | ROC AUC: 0.815 | EER: 0.255 @ th=0.152
model: svm        || f1: 0.516 | prec: 0.412 recall: 0.690 | ROC AUC: 0.865 | EER: 0.200 @ th=0.166
model: forest     || f1: 0.627 | prec: 0.485 recall: 0.887 | ROC AUC: 0.890 | EER: 0.141 @ th=0.178
model: gbc        || f1: 0.765 | prec: 0.670 recall: 0.890 | ROC AUC: 0.878 | EER: 0.147 @ th=0.075
model: xgb        || f1: 0.784 | prec: 0.691 recall: 0.905 | ROC AUC: 0.886 | EER: 0.165 @ th=0.080


# Stacking

Let's create a stacked model on top of the base learners above and see if it improves at least any of the classification metrics

In [31]:
# let's not add kNN classifier as the weakest of all the models above
estimators_to_stack = [
    ("forest", best_rfc),
    ("gbc", best_gb),
    ("xgb", best_xgb),
    ("svm", best_svc),
    ("logreg", best_logreg),
]

In [32]:
stacked_model = StackingClassifier(
    estimators=estimators_to_stack,
    final_estimator=GradientBoostingClassifier(),
    cv=StratifiedKFold(4, shuffle=True),
)
stacked_model.fit(X_train, y_train)
stacked_metrics = get_binary_classification_metrics(stacked_model, X_test, y_test)

print_report(stacked_metrics, model_name="stacked")

model: stacked    || f1: 0.779 | prec: 0.763 recall: 0.796 | ROC AUC: 0.904 | EER: 0.176 @ th=0.031


## We can do cross-validation here!

In [33]:
skf_stacked = StratifiedKFold(5, shuffle=True, random_state=42)
stacked_model = StackingClassifier(
    estimators=estimators_to_stack,
    final_estimator=GradientBoostingClassifier(),
    cv=StratifiedKFold(4, shuffle=True),
)
# Gradient Boosting
parameters_stacking = {
    'final_estimator__n_estimators': [50, 100, 150, 200],
    'final_estimator__learning_rate': [0.05, 0.1, 0.3],
    'final_estimator__min_samples_leaf': [1, 3, 5],
    'final_estimator__max_depth': [2, 3],
}
gcv_stacked = GridSearchCV(stacked_model, parameters_stacking, **grid_search_params)
gcv_stacked.fit(X_train, y_train)

Fitting 5 folds for each of 72 candidates, totalling 360 fits


GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=42, shuffle=True),
             estimator=StackingClassifier(cv=StratifiedKFold(n_splits=4, random_state=None, shuffle=True),
                                          estimators=[('forest',
                                                       RandomForestClassifier(max_depth=10,
                                                                              max_features=10,
                                                                              min_samples_leaf=3,
                                                                              n_jobs=-1,
                                                                              oob_score=True,
                                                                              random_state=42)),
                                                      ('gbc',
                                                       GradientBoostingClassifier(learning_rate=0....
                                                                        LogisticRegression(C=0.1,
                                                                                           penalty='l1',
                                                                                           random_state=42,
                                                                                           solver='liblinear'))]))],
                                          final_estimator=GradientBoostingClassifier()),
             n_jobs=-1,
             param_grid={'final_estimator__learning_rate': [0.05, 0.1, 0.3],
                         'final_estimator__max_depth': [2, 3],
                         'final_estimator__min_samples_leaf': [1, 3, 5],
                         'final_estimator__n_estimators': [50, 100, 150, 200]},
             scoring='roc_auc', verbose=1)

In [34]:
best_stacked = gcv_stacked.best_estimator_
best_params_stacked = gcv_stacked.best_params_
best_params_stacked

{'final_estimator__learning_rate': 0.1,
 'final_estimator__max_depth': 2,
 'final_estimator__min_samples_leaf': 5,
 'final_estimator__n_estimators': 200}

In [35]:
best_stacked_metrics = get_binary_classification_metrics(best_stacked, X_test, y_test)
print_report(best_stacked_metrics, model_name="best stacked")

model: best stacked || f1: 0.811 | prec: 0.773 recall: 0.852 | ROC AUC: 0.913 | EER: 0.177 @ th=0.034
